<a href="https://colab.research.google.com/github/mohithxoxo/NLP/blob/master/BERT_with_ktrain(Sentimental_Classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

KTrain is a wrapper for BERT technique of text classification

In [ ]:
!pip install ktrain
from ktrain import text 
import ktrain
import pandas as pd

In [6]:
train = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
test = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/test.tsv', delimiter='\t', header=None)

Extracting training and validation dataset from module

In [7]:
train = train.rename(columns={0:'text',1:'label'})
test = test.rename(columns={0:'text',1:'label'})

In [8]:
trn, val, preproc = text.texts_from_df(train,'text', label_columns=['label'], val_df=test,preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


Fetching the model and learner instance

In [15]:
model = text.text_classifier('bert', trn, preproc=preproc)
learner = ktrain.get_learner(model,train_data=trn, val_data=val, batch_size=6)

Is Multi-Label? False
maxlen is 400
done.


Start to train the model

In [16]:
learner.fit_onecycle(2e-5, 2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
1154/1154 [==============================] - 1657s 1s/step - loss: 0.3844 - accuracy: 0.8231 - val_loss: 0.3598 - val_accuracy: 0.8440
Epoch 2/2
1154/1154 [==============================] - 1657s 1s/step - loss: 0.1628 - accuracy: 0.9395 - val_loss: 0.2075 - val_accuracy: 0.9121


In [17]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['not_label', 'label']

In [18]:
#Positive
predictor.predict('happy')

'label'

In [19]:
#Negative
predictor.predict('sad')

'not_label'

In [20]:
# let's save the predictor for later use
predictor.save('/content/my_model')
# reload the predictor
mymodel = ktrain.load_predictor('/content/my_model')
# make a prediction on the same document to verify it still works
mymodel.predict('sad')

'not_label'

In [21]:
# Saving Model for deployment
model_json = learner.model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

learner.model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
